In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from transformers import pipeline
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
examples = [
    {
        "question":"What are some products I should buy if I'm already buying shoes",
        "answer":'''
        Socks
        Shoe Polish
        Insoles
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying laptops",
        "answer":'''
        Mouse
        Laptop Case
        Keyboard
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying school bags",
        "answer":'''
        Books
        Pens
        Pencils
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying beds",
        "answer":'''
        Blankets
        Pillows
        Bedsheet
        '''
    },
    {
        "question":"What are some products I should buy if I'm already buying toys",
        "answer":'''
        Chocolates
        Arts and Craft
        Board Games
        '''
    }
]

In [ ]:
# instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
# system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

# template = get_prompt(instruction, system_prompt)

# prompt = PromptTemplate(
#     input_variables=["chat_history", "user_input"], template=template
# )
# memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
# example_template = """
# User: {question}
# AI: {answer}
# """
# system_prompt =""" The assistant is a world class recommender of products. The assisstant gives 3 most appropriate items most compatible with product in question
# Here are some examples of excerpts between user and the assiatant. Give only 3 products as the output"""

# template = get_prompt(example_template, system_prompt)

# example_prompt = PromptTemplate(
#     input_variables=["question", "answer"],
#     template=template
# )


In [5]:
'''MY METHOD'''

example_template = """
User: {question}
AI: {answer}
"""
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template=example_template
)

prefix = """ The assisstant is a world class recommender of products. The assisstant gives 3 most appropriate items most compatible with product in question
Here are some examples of excerpts between user and the assiatant. Give only 3 products as the output"""



In [6]:
suffix = """
User: {question}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["question"],
    example_separator="\n\n"
)

In [7]:
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

In [ ]:
#meta-llama/Llama-2-13b-chat-hf

In [8]:
  model = "meta-llama/Llama-2-7b-chat-hf"

  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                            use_auth_token=True,)

  model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                              device_map='auto',
                                              use_auth_token=True,
                                              offload_folder="save_folder"
                                              #  load_in_8bit=True,
                                              #  load_in_4bit=True
                                              )

  # pipeline = transformers.pipeline(
  #     "text-generation",
  #     model=model,
  #     torch_dtype=torch.float16,
  #     device_map="auto",
  # )

  pipe = pipeline("text-generation",
                  model=model,
                  tokenizer= tokenizer,
                  device_map="auto",
                  max_new_tokens = 512,
                  do_sample=True,
                  top_k=30,
                  num_return_sequences=1,
                  eos_token_id=tokenizer.eos_token_id,
                  )

  llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:628: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [9]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template,verbose=False,memory=memory)

In [ ]:
chain.predict(question="What are some products I should buy if I'm already buying headphones")